In [2]:
from azureml.core.compute import AmlCompute
from azureml.core.runconfig import CondaDependencies, RunConfiguration
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.core import Workspace
from azure.storage.blob import BlockBlobService
import pandas as pd
import azureml
import io
import os
print("SDK version:", azureml.core.VERSION)

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azure-mgmt-resource 18.0.0 (c:\programdata\anaconda3\lib\site-packages), Requirement.parse('azure-mgmt-resource<15.0.0,>=1.2.1'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azure-mgmt-resource 18.0.0 (c:\programdata\anaconda3\lib\site-packages), Requirement.parse('azure-mgmt-resource<15.0.0,>=1.2.1'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azure-mgmt-resource 18.0.0 (c:\programdata\anaconda3\lib\site-packages), Requirement.parse('azure-mgmt-resource<15.0.0,>=1.2.1'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml

SDK version: 1.19.0


In [3]:
# connection instance creation
container_name = 'spardata'
blobstorageaccount = 'sparmlstorage'
block_blob_service = BlockBlobService(account_name=blobstorageaccount,
                                      account_key=os.environ['account_key'])

In [4]:
# source directory
source_directory = 'scripts'
print('Sample scripts will be created in {} directory.'.format(source_directory))

Sample scripts will be created in scripts directory.


In [5]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id=os.environ['tenant_id'])

ws = Workspace(subscription_id=os.environ['subscription_id'],
               resource_group="mlservice",
               workspace_name="mlservice",
               auth=interactive_auth)
print("Found workspace {} at location {}".format(ws.name, ws.location))

Found workspace mlservice at location westeurope


In [6]:
cts = ws.compute_targets
for ct in cts:
    print(ct)

spar
admin8


In [14]:
# retriving stores list
with io.BytesIO() as input_blob:
    block_blob_service.get_blob_to_stream(container_name='planning',
                                        blob_name = r'planning_stores.csv',
                                        stream = input_blob)
    input_blob.seek(0)
    list_blobs = pd.read_csv(input_blob)
list_blobs = list_blobs['ObjCode'].values


# creating 30 parallel threads
streams = 30
strings = []

for i in range(streams):
    strings.append([]) 

for i in range(len(list_blobs)):
    k = i%streams
    strings[k].append(list_blobs[i])
    
for i in range(len(strings)):
    strings[i] = str(strings[i]).replace('[', '').replace(']', '').replace(' ', '')

In [18]:
len(strings)

30

### The old pipeline deleting 

In [12]:
from azureml.pipeline.core import PublishedPipeline
published_pipeline = PublishedPipeline.get(ws, '{}'.format('4b101d0d-5f4a-45d3-a95a-1db81431e347'))
published_pipeline.disable()

### Creation of the new pipeline

In [20]:
compute_target = AmlCompute(ws, "support-promo")

conda_dependencies = CondaDependencies.create(
    pip_packages=["numpy", "scipy", "pandas==0.25.1",  "azure-storage-blob==2.1.0", "catboost", "xlrd==1.2.0", "scikit-learn==0.24.1"], python_version="3.6.7"
) # "catboost",
run_config = RunConfiguration(conda_dependencies=conda_dependencies)
run_config.environment.docker.enabled = True
run_config.source_directory_data_store = "workspaceblobstore"


steps = []

step1 = PythonScriptStep(allow_reuse=False,
        name="collect",
        script_name="collect.py",
        compute_target=compute_target,
        runconfig=run_config,
        source_directory = "scripts")

step2 = PythonScriptStep(allow_reuse=False,
        name="collect_code",
        script_name="collect_code.py",
        compute_target=compute_target,
        runconfig=run_config,
        source_directory = "scripts")


score_step_temp_promo_deleting = PythonScriptStep(allow_reuse=False,
    name="promo_deleting_step",
    script_name="temp_promo_deleting.py",
    compute_target=compute_target,
    runconfig=run_config,
    source_directory = "temp_promo_deleting")



for stores in list_blobs:
    
    score_step1 = PythonScriptStep(allow_reuse=False,
        name="discount_scoring",
        script_name="discount_updating.py",
        arguments=["--stores", "{}".format(stores)],
        compute_target=compute_target,
        runconfig=run_config,
        source_directory = "scripts")
    
    score_step = PythonScriptStep(allow_reuse=False,
        name="prediction_scoring",
        script_name="average.py",
        arguments=["--stores", "{}".format(stores)],
        compute_target=compute_target,
        runconfig=run_config,
        source_directory = "scripts")
    
    score_step.run_after(score_step1)
    step1.run_after(score_step) 
    step2.run_after(step1)
#     steps.append(score_step1)
#     steps.append(score_step)

score_step_temp_promo_deleting.run_after(step1)
score_step_temp_promo_deleting.run_after(step2) 

steps.append(score_step_temp_promo_deleting)
print(len(steps))
pipeline = Pipeline(workspace=ws, steps=steps)

In [23]:
pipeline_name = "promo_tables_21_04_2023"
print(pipeline_name)

published_pipeline = pipeline.publish(
    name=pipeline_name, 
    description=pipeline_name)
print("Newly published pipeline id: {}".format(published_pipeline.id))

promo_tables_21_04_2023
Created step promo_deleting_step [b2c1db04][1ce94d4e-f841-4c65-a998-1cc4b6a29eac], (This step will run and generate new outputs)Created step collect [7ee3e5ba][f0ee63ef-d012-455e-8df6-ec1fe1946e40], (This step will run and generate new outputs)

Created step prediction_scoring [1abc8cec][f8e66eb6-4b74-4a0b-8e4a-ee8afe95901c], (This step will run and generate new outputs)Created step discount_scoring [633822f9][6f156d25-d61b-4beb-9b15-72948c2c2312], (This step will run and generate new outputs)Created step prediction_scoring [050e76c6][36821924-b343-4ba6-919c-a9129e0b6746], (This step will run and generate new outputs)

Created step discount_scoring [623d54b9][7258bc46-5db3-49b3-8ebf-16561d10bf5d], (This step will run and generate new outputs)

Created step prediction_scoring [4015d353][bb3c98da-c74a-4565-b690-3f7fbb2fe20b], (This step will run and generate new outputs)
Created step discount_scoring [de687f81][7e29cbfc-c31e-49fb-a773-2a6d406aa7fd], (This step wil

### Scheduling of the new pipeline

In [24]:
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule

recurrence = ScheduleRecurrence(frequency="Week", interval=1, 
                                week_days = ["Monday"],
                                hours=[19], minutes=[10]) 


schedule = Schedule.create(workspace=ws, name="Promo_Schedule",
                           pipeline_id='{}'.format(published_pipeline.id), 
                           experiment_name='promo_tables',
                           recurrence=recurrence,
                           wait_for_provisioning=True,
                           description="Schedule Run")

print("Created schedule with id: {}".format(schedule.id))

Provisioning status: Completed
Created schedule with id: 26e3ab1b-5234-4e53-8c63-09e60e448e6c


In [8]:
# list of schedules
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule
schedules = Schedule.list(ws, active_only=True)
print(schedules)

[]


In [11]:
# Deleting a schedule
fetched_schedule = Schedule.get(ws, '{}'.format('ba7763cc-0373-49da-b318-cfdbbcc2d309'))
fetched_schedule.disable()